In [ ]:
# 필요한 라이브러리 설치
!pip install yt-dlp
!pip install pydub
!pip install SpeechRecognition

In [ ]:
import yt_dlp
import os
import glob
from pydub import AudioSegment
import speech_recognition as sr
from math import floor

In [ ]:
def download_audio_from_youtube(url, out_template='downloaded_audio.%(ext)s'):
    """
    yt-dlp로 원본 포맷(예: webm)을 그대로 저장.
    반환: 저장된 파일 경로 (확장자 포함) 또는 None
    """
    ydl_opts = {
        'format': 'bestaudio/best',
        'outtmpl': out_template,
        'quiet': False,
        'no_warnings': True,
    }
    try:
        with yt_dlp.YoutubeDL(ydl_opts) as ydl:
            ydl.download([url])
        # 파일명은 out_template에서 %()s 부분에 실제 확장자가 들어가므로 glob으로 찾음
        candidates = glob.glob('downloaded_audio.*')
        if candidates:
            # 최신 생성 파일 반환
            candidates.sort(key=os.path.getmtime, reverse=True)
            return candidates[0]
        return None
    except Exception as e:
        print("다운로드 에러:", e)
        return None

In [ ]:
def convert_to_wav(input_filepath, wav_path='converted_audio.wav'):
    """
    pydub의 자동 포맷 감지를 사용하여 입력파일을 WAV로 변환.
    """
    try:
        audio = AudioSegment.from_file(input_filepath)  # format 자동 감지
        audio.export(wav_path, format='wav')
        return wav_path
    except Exception as e:
        print("변환 에러:", e)
        return None

In [ ]:
def split_audio(wav_path, chunk_length_s=30):
    """
    WAV 파일을 chunk_length_s 초 단위로 나눔.
    반환: [(start_s, end_s, chunk_path), ...]
    """
    audio = AudioSegment.from_file(wav_path, format='wav')
    duration_ms = len(audio)
    chunk_ms = chunk_length_s * 1000
    chunks = []
    idx = 0
    start_ms = 0
    while start_ms < duration_ms:
        end_ms = min(start_ms + chunk_ms, duration_ms)
        chunk = audio[start_ms:end_ms]
        chunk_path = f"chunk_{idx:04d}.wav"
        chunk.export(chunk_path, format='wav')
        chunks.append((start_ms//1000, end_ms//1000, chunk_path))
        idx += 1
        start_ms += chunk_ms
    return chunks

In [ ]:
def transcribe_chunks_and_save(chunks, out_textfile='transcript.txt', language='ko-KR'):
    """
    chunks: list of (start_s, end_s, chunk_path)
    speech_recognition의 Google API로 순차적으로 인식하고 파일에 append.
    """
    r = sr.Recognizer()
    # 결과 파일 덮어쓰기 모드로 시작
    with open(out_textfile, 'w', encoding='utf-8') as f_out:
        for start_s, end_s, chunk_path in chunks:
            try:
                with sr.AudioFile(chunk_path) as source:
                    audio_data = r.record(source)
                    # Google Web API 사용(인터넷 필요). 요금/제한 주의.
                    text = r.recognize_google(audio_data, language=language)
            except sr.UnknownValueError:
                text = "[음성 인식 불가]"
            except sr.RequestError as e:
                text = f"[API 요청 실패: {e}]"
            except Exception as e:
                text = f"[오류: {e}]"

            # 타임스탬프 붙여 저장
            timestamp = f"{format_time(start_s)} --> {format_time(end_s)}"
            f_out.write(f"{timestamp}\n{text}\n\n")
            print(f"[{timestamp}] {text}")

In [ ]:
def format_time(s):
    h = s // 3600
    m = (s % 3600) // 60
    sec = s % 60
    if h:
        return f"{h:02d}:{m:02d}:{sec:02d}"
    else:
        return f"{m:02d}:{sec:02d}"

In [ ]:
def cleanup_temp(files):
    for f in files:
        try:
            os.remove(f)
        except:
            pass

In [ ]:
def main_batch(youtube_url):
    print("다운로드 시작...")
    downloaded = download_audio_from_youtube(youtube_url, out_template='downloaded_audio.%(ext)s')
    if not downloaded:
        print("다운로드 실패")
        return
    print("다운로드 완료:", downloaded)

    wav = convert_to_wav(downloaded, wav_path='converted_audio.wav')
    if not wav:
        print("WAV 변환 실패")
        return
    print("WAV 변환 완료:", wav)

    print("오디오 분할...")
    chunks = split_audio(wav, chunk_length_s=30)
    print(f"생성된 청크 개수: {len(chunks)}")

    print("음성인식 시작...")
    transcribe_chunks_and_save(chunks, out_textfile='transcript.txt', language='ko-KR')
    print("완료: transcript.txt 생성됨")

    # 임시파일 정리
    cleanup_list = [downloaded, wav] + [c[2] for c in chunks]
    cleanup_temp(cleanup_list)
    print("임시파일 삭제 완료.")

In [ ]:
if __name__ == '__main__':
    url = input("YouTube 영상 URL을 입력하세요: ").strip()
    main_batch(url)